In [29]:
import pymysql as sql
from pprint import pprint
import hashlib


In [18]:
# db 불러오기
conn = sql.connect(host= "localhost", user="root", password='1234', db='cache_db', charset='utf8')
table_name = "cache"
require_min_length = 5

In [15]:
conn

In [30]:
def select_all_data():
    cursor = conn.cursor()
    cursor.execute(f"select * from {table_name};")

    cur_db = cursor.fetchall()
    for d in cur_db:
        pprint(d)

In [31]:
select_all_data()

(1,
 datetime.datetime(2025, 1, 14, 11, 50, 46),
 '결제 후 주문 취소가 가능한가요?',
 'b65de77b98474d2b6b46db080add07f0,e4d477a414f3783aa6b9a762d34bf195,fdfce5acb9c600b8f2ffb7ced6f3d719',
 'Answer : 네, 고객님.\n'
 ' 결제 후 주문 취소 가능 여부는 상품 배송 여부에 따라 달라집니다.\n'
 ' 배송 전이라면 주문 취소가 가능하지만, 배송이 시작된 경우에는 취소가 어려울 수 있습니다.\n'
 ' 정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다.\n'
 ' 주문번호 확인 후, 취소 가능 여부와 절차를 안내해 드리겠습니다.\n'
 '\n'
 '\n'
 '\n'
 '**조건별 대응 방법**\n'
 '\n'
 '* **배송 전:** 고객님의 주문번호를 확인하여 주문 취소를 진행해 드립니다.\n'
 ' 취소 완료 후 결제 취소까지는 카드사별로 3~7일(주말 및 공휴일 제외) 정도 소요될 수 있으며, 자세한 내용은 해당 카드사에 '
 '문의하시는 것이 좋습니다.\n'
 '\n'
 '\n'
 '* **배송 중 또는 배송 완료:** 주문 취소는 불가능하며, 배송 완료 후 반품을 접수하셔야 합니다.\n'
 ' 단, 단순 변심으로 인한 반품 시 왕복 배송비가 부과될 수 있다는 점 양해 부탁드립니다.\n'
 ' 반품 절차에 대한 안내를 드리겠습니다.\n'
 '\n'
 '\n'
 '* **주문 취소 철회 요청:** 이미 주문 취소를 완료하신 경우에는 취소를 철회할 수 없습니다.\n'
 ' 재구매를 부탁드립니다.\n'
 '\n'
 '\n'
 '* **기타:** 제공된 정보만으로는 답변이 어려운 경우, "문의주신 내용은 확인이 필요하여 지금 답변드리기 어려울 것 같습니다.\n'
 ' 번거로우시겠지만 확인 후에 다시 연락드려도 괜찮을까요?" 라고 답변합니다.\n'
 '\n'
 '.')
(2,
 datetime.datetime(2025, 1,

In [9]:
def validation_data(query):

    return len(query) > require_min_length
    

In [12]:
# 데이터 삽입

def insert_data(query):
    if not validation_data(query):
        print(f"최소 {require_min_length}자 이상의 질문을 입력해 주세요")
        return

    try:
        with conn.cursor() as cur:
            query = "insert into cache (created_at, query, hashed_docs, created_answer) values (%s, %s, %s, %s);"
            cur.execute("use cache")

            cur.execute(query, (
                
            ))

            conn.commit()

    finally:
        conn.close()

In [37]:
# DB에 현재 md5 정보와 일치하는 문서가 있는지 확인
def is_exist_doc(query_md5):
    try:
        with conn.cursor() as cur:
            find_query_md5 = f"select * from {table_name} where hashed_docs = '{query_md5}';"
            print(find_query_md5)
            cur.execute(find_query_md5)
            
            exist_doc = cur.fetchall()
            return exist_doc
            
    except Exception as e:
        print(e)
        

def query_to_md5(query):
    return ",".join(sorted([hashlib.md5(result.page_content.encode()).hexdigest() for result in database.similarity_search(query=query, k=3)]))


In [62]:
def get_exist_llm_answer(query):
    docs = is_exist_doc(query_md5)
    
    # 일치하는 docs가 없다면
    if len(docs) == 0:
        # generate_and_store_llm()
        pass
    
    else:
        answer = docs[0][4]
        print(answer)
        

In [57]:
test_md5 = "5aec9b0cbb8e768576a0bc99157cbb8a,72f328e328ed93624991283a5e2e3c09,de14d9dd57ef5fcc9a3f113baea01aa7"

In [63]:
doc = is_exist_doc(test_md5)

select * from cache where hashed_docs = '5aec9b0cbb8e768576a0bc99157cbb8a,72f328e328ed93624991283a5e2e3c09,de14d9dd57ef5fcc9a3f113baea01aa7';


In [64]:
get_exist_llm_answer(test_md5)

select * from cache where hashed_docs = '5aec9b0cbb8e768576a0bc99157cbb8a,72f328e328ed93624991283a5e2e3c09,de14d9dd57ef5fcc9a3f113baea01aa7';
query:  주문 후 배송 주소를 변경할 수 있나요?
Answer : 네, 주문 후 배송 주소 변경 가능 여부는 주문 시점과 상품 상태에 따라 달라집니다.
 정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다.
 주문 직후라면 변경이 가능하지만, 이미 상품이 출고되었거나 선물하기 상품인 경우 변경이 어려울 수 있습니다.



**각 조건별 대응 방법**

* **주문 직후 (상품 미출고):** `주문번호 확인 후 변경 가능 여부를 확인해 드리겠습니다.
 마이쿠팡 > 주문목록 > 주문상세보기에서 `받는사람 정보`의 `받는주소` 변경을 시도해 보시고, 변경이 안 되면 바로 연락 주세요.
 최대한 도와드리겠습니다.
`

* **상품 출고 후:** `죄송합니다만, 상품이 이미 출고되어 배송지 변경이 불가능합니다.
 불편을 드려 죄송합니다.
`

* **선물하기 상품:** `죄송합니다만, 선물하기 상품은 배송지 변경이 불가능합니다.
 수취인이 배송지를 변경할 수도 없으므로, 쿠팡 고객센터(1577-7011)로 연락하셔서 취소 및 반품을 접수하시는 것이 좋겠습니다.
`
.
